In [ ]:
!pip install -q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
from langchain_community.chat_models import ChatCohere

llm = ChatCohere()

In [ ]:
import os
os.makedirs("instructions")

In [ ]:
!mv *.md instructions

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [ ]:
from crewai_tools import BaseTool

In [ ]:
class SentimentAnalysis(BaseTool):
  name: str= "Sentiment Analysis Tool"
  description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

  def _run(self, text:str)->str:
    return "positive"

In [ ]:
sentement_analysis_tool=SentimentAnalysis()

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentement_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [ ]:
crew=Crew(
    agents=[sales_rep_agent, lead_sales_rep_agent],
    tasks=[lead_profiling_task, personalized_outreach_task],
    verbose=2
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by searching for information about DeepLearningAI to gather data for the report.
Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company"}
Observation: ```
The DeepLearningAI Organization: Transforming AI Education
DeepLearningAI is a pioneering organization in the field of artificial intelligence (AI) education, founded by Andrew Ng, a renowned computer

In [ ]:
from IPython.display import Markdown
Markdown(result)

Subject: CrewAI's Solutions for DeepLearningAI's Product Launch


Dear Andrew Ng,

I hope this email finds you well. As the CEO of DeepLearningAI, I know you are dedicated to transforming AI education and empowering individuals and organizations to thrive in the AI-driven future.

At CrewAI, we share a similar vision of enabling organizations to harness the power of AI effectively. We believe that our solutions can significantly contribute to DeepLearningAI's mission and recent product launch.

**AI Talent Development:**
CrewAI can collaborate with DeepLearningAI to create industry-specific AI training programs, helping organizations develop their AI talent. With our expertise, we can ensure that the programs are tailored to the unique needs of various sectors, including healthcare, finance, and manufacturing.

**AI-Powered Learning Tools:**
Our AI capabilities can enhance DeepLearningAI's platform with features like personalized learning paths, AI-assisted coding, and automated assessments. These tools will not only improve the learning experience but also ensure that learners acquire practical skills relevant to their industries.

**Industry-Specific Courses:**
CrewAI's industry knowledge can contribute to developing courses tailored to specific sectors. By leveraging our insights, DeepLearningAI can create courses that cater to the unique challenges and opportunities in various industries, ensuring learners gain relevant and actionable knowledge.

We believe that by partnering with DeepLearningAI, we can create a powerful synergy, enabling organizations to embrace the potential of AI more effectively. Our solutions are designed to support your recent product launch and future goals, ensuring that learners receive the best possible education in AI.

I would be delighted to schedule a call to discuss these opportunities further and explore how CrewAI can contribute to DeepLearningAI's success.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

---

Subject: CrewAI's Solutions for DeepLearningAI's Product Launch


Dear Mr. Ng,

Congratulations on DeepLearningAI's recent product launch! As an industry leader in AI education, your commitment to democratizing AI knowledge is truly inspiring.
At CrewAI, we believe that our solutions can further support DeepLearningAI's mission and recent achievements. Here's how:

**AI-Powered Learning Experience:**
Our AI expertise can enhance DeepLearningAI's platform with advanced features like AI-assisted coding, personalized learning paths, and automated assessments. These tools will not only improve the learning experience but also ensure that learners acquire practical, industry-relevant skills.

**Industry-Specific Courses:**
With our deep industry knowledge, we can collaborate on developing courses tailored to specific sectors. By leveraging our insights, DeepLearningAI can create courses that cater to the unique challenges and opportunities in various industries, ensuring learners gain actionable knowledge.

**AI Talent Development:**
CrewAI can partner with DeepLearningAI to create industry-specific AI training programs, helping organizations develop their AI talent. Our expertise in various sectors will ensure that the programs are relevant and effective.

We believe that by combining DeepLearningAI's educational expertise with CrewAI's industry insights and AI capabilities, we can create a powerful synergy. Together, we can enable organizations to embrace the potential of AI more effectively.

I would be honored to schedule a call to discuss these opportunities further and explore how CrewAI can contribute to DeepLearningAI's success.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

---

Subject: CrewAI's Solutions for DeepLearningAI's Product Launch

Dear Andrew,

I am reaching out to you on behalf of CrewAI, a company dedicated to empowering organizations with AI solutions. We are impressed by DeepLearningAI's recent product launch and believe that our solutions can significantly contribute to your mission.

**AI-Powered Learning:**
CrewAI's AI capabilities can enhance DeepLearningAI's platform with features like AI-assisted coding, personalized learning paths, and automated assessments. These tools will ensure that learners receive a personalized and engaging learning experience, helping them acquire practical skills.

**Industry-Specific Courses:**
With our industry expertise, we can collaborate on developing courses tailored to specific sectors. By leveraging our insights, DeepLearningAI can create courses that cater to the unique needs of various industries, ensuring learners gain relevant and actionable knowledge.

**AI Talent Development:**
CrewAI can partner with DeepLearningAI to create industry-specific AI training programs, helping organizations develop their AI talent. Our expertise will ensure that the programs are effective and aligned with the unique needs of different sectors.

We believe that by aligning our solutions, we can create a powerful synergy, enabling organizations to embrace the potential of AI more effectively. Our goal is to support DeepLearningAI's mission and ensure learners receive the best possible education in AI.

I would be delighted to schedule a call to discuss these opportunities further and explore how CrewAI can contribute to DeepLearningAI's success.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI